In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
from ipywidgets import interact
import warnings
%matplotlib inline

# load data observasi / data lapangan
d_obs = np.loadtxt("DataGravityLapanganJumatLatihan.txt")

# membuat variabel baru
xData = d_obs[:,0]
gData = d_obs[:,1]

rng = np.random.default_rng()
noise = 0.005

gData = gData + noise*rng.normal(size=21)

In [2]:
@interact
def testing(rho=2.67, jari=20, zSource=47, xSource=200):
    # input paramater
    rho = rho # gr/cc
    jari = jari # meter
    zSource = zSource # meter
    xSource = xSource # meter

    # properti stasiun
    x0 = 0 # meter
    xn = np.max(xData) # meter
    nx = xn / len(xData) + 1 # meter

    def gravsphere(rho, jari, zSource, xSource, x):
        G = 6.67e-11 # konstanta gravitasi universal, SI
        gravity = G*rho*(4/3)*np.pi*(jari**3)*(zSource/(zSource**2 + (xSource-x)**2)**(3/2))*1e8

        return gravity

    n = len(xData)

    xStasiun = np.zeros(n)
    gz = np.zeros(n)
    x = np.zeros(n)

    for i in range(n):
        x = (i) * nx
        xStasiun[i] = (i) * nx
        gz[i] = gravsphere(rho, jari, zSource, xSource, x)

    # menghitung misfit
    misfit = round(np.sum((gData-gz)**2),4)

    ## plot anomaly respons
    plt.figure(figsize=(8,8))
    plt.subplot(2,1,1)
    plt.plot(xStasiun, gz, 'ro')
    plt.plot(xData, gData, 'b-')
    plt.ylabel("gravity anomaly (mGal)")
    plt.xlim(0,500)
    plt.ylim(0,0.3)
    plt.grid()
    plt.legend(['calculated', 'observation'])
    plt.title("Forward Modelling Gravity (Sphere)")
    plt.text(400,0.15, f"misfit: {misfit}",bbox = dict(facecolor = 'white'))

    q = np.linspace(0,2*np.pi,500)
    plt.subplot(2,1,2)
    xSphere = jari*np.cos(q)+xSource
    zSphere = jari*np.sin(q)-zSource
    plt.ylabel("depth (m)")
    plt.xlim(0,500)
    plt.ylim(-150,75)
    plt.xlabel("distance (m)")
    plt.fill(xSphere, zSphere, 'r')
    plt.grid()
    plt.text(xSource+30,-zSource, f"densitas: {rho} gr/cc",bbox = dict(facecolor = 'white'))
    plt.show()

interactive(children=(FloatSlider(value=2.67, description='rho', max=8.01, min=-2.67), IntSlider(value=20, des…